In [195]:
import pandas as pd
import time
import datetime

df = pd.read_csv ('output/real-west-all-terminals/states.csv', header = 0)
arrivals = df.groupby(by=['callsign']).filter(lambda x: (x['takeoff'].max() == False).any())
arrivals = arrivals[['time', 'callsign']]
# print(arrivals[arrivals['callsign'] == 'AA1260-D'])
# print(len(arrivals))
arrivals['time'] = arrivals['time'].astype('datetime64[ns]')

arrival_taxi_time = arrivals.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
arrival_taxi_time['taxi_time'] = arrival_taxi_time['time']['max'] - arrival_taxi_time['time']['min']
arrival_taxi_time['taxi_time_seconds'] = arrival_taxi_time['taxi_time'].dt.total_seconds()
print(len(arrival_taxi_time))
# print(arrival_taxi_time[:10])


sort = arrival_taxi_time.sort_values(by=['taxi_time_seconds'], ascending=False)
print(sort[10:]["taxi_time_seconds"].mean())

# avg_total_ticks = arrival_taxi_time["taxi_time_seconds"].mean()
# print(avg_total_ticks)
# print(arrivals)


76
722.7272727272727


In [303]:
df_before = pd.read_csv ('/Users/lolaliu/Desktop/airport-simulation/output/real-west-all-terminals/states.csv', header = 0)
arrivals_before = df_before.groupby(by=['callsign']).filter(lambda x: x['takeoff'].max() == False)
arrivals_before = arrivals_before[["time", "callsign"]]
# print(arrivals[arrivals['callsign'] == 'AA1260-D'])
# print(len(arrivals_before))
arrivals_before['time'] = arrivals_before['time'].astype('datetime64[ns]')

arrival_taxi_time_before = arrivals_before.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
arrival_taxi_time_before['taxi_time'] = arrival_taxi_time_before["time"]["max"] - arrival_taxi_time_before["time"]["min"]
arrival_taxi_time_before['taxi_time_seconds'] = arrival_taxi_time_before['taxi_time'].dt.total_seconds()
print(len(arrival_taxi_time_before))

sort_before = arrival_taxi_time_before.sort_values(by=[('time', 'max')], ascending=False)
# print(sort_before[10:]["taxi_time_seconds"].mean())
print(arrival_taxi_time_before[:10])

# avg_total_ticks_before = arrival_taxi_time_before["taxi_time_seconds"].mean()
# print(avg_total_ticks_before)

87
   callsign                time                           taxi_time  \
                            min                 max                   
0   4O821-D 2021-11-11 10:46:00 2021-11-11 23:55:30 0 days 13:09:30   
1  AA1322-A 2021-11-11 07:58:00 2021-11-11 08:11:30 0 days 00:13:30   
2   AA152-A 2021-11-11 08:43:00 2021-11-11 14:29:00 0 days 05:46:00   
3  AA1880-D 2021-11-11 12:23:00 2021-11-11 23:55:30 0 days 11:32:30   
4   AC560-A 2021-11-11 08:35:00 2021-11-11 08:56:00 0 days 00:21:00   
5   AC738-D 2021-11-11 12:13:00 2021-11-11 23:55:30 0 days 11:42:30   
6   AC754-D 2021-11-11 22:59:00 2021-11-11 23:55:30 0 days 00:56:30   
7   AC760-D 2021-11-11 09:45:00 2021-11-11 23:55:30 0 days 14:10:30   
8   AC776-D 2021-11-11 22:56:00 2021-11-11 23:55:30 0 days 00:59:30   
9  AC8578-D 2021-11-11 11:54:00 2021-11-11 23:55:30 0 days 12:01:30   

  taxi_time_seconds  
                     
0           47370.0  
1             810.0  
2           20760.0  
3           41550.0  
4           

In [168]:
only_before = arrival_taxi_time_before.merge(arrival_taxi_time, on=['callsign'], 
                   how='left', indicator=True)
only_before = only_before[only_before['_merge'] == 'left_only']
print(len(only_before))
print(only_before['callsign'][:5])

only_after = arrival_taxi_time.merge(arrival_taxi_time_before, on=['callsign'], 
                   how='left', indicator=True)
only_after = only_after[only_after['_merge'] == 'left_only']
print(len(only_after))
print(only_after['callsign'][:5])

46
2      AA16-D
4     AC760-D
5    AC8578-D
6    AC8744-D
7     AI174-D
Name: callsign, dtype: object
31
0     4O820-A
1    AA1260-D
4     AA785-A
8    AS1925-A
9    AS1926-D
Name: callsign, dtype: object


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [193]:
overlap_arrivals = pd.merge(arrival_taxi_time_before, arrival_taxi_time, on="callsign")

print("arrival")
print("before: " + str(overlap_arrivals['taxi_time_seconds_x'].mean()))
print("after: " + str(overlap_arrivals['taxi_time_seconds_y'].mean()))

merge_sorted = overlap_arrivals.sort_values(by=['taxi_time_seconds_y', 'taxi_time_seconds_x'], ascending=False)
print(merge_sorted)
# print(len(overlap_arrivals))
# print(df_all[:5])

arrival
before: 3302.6666666666665
after: 982.0
    callsign              time_x                         taxi_time_x  \
                             min                 max                   
22   UA234-D 2021-11-09 08:32:00 2021-11-09 11:56:00 0 days 03:24:00   
23  UA2356-A 2021-11-09 07:38:30 2021-11-09 08:32:00 0 days 00:53:30   
24   UA256-A 2021-11-09 07:58:00 2021-11-09 11:56:00 0 days 03:58:00   
4   AS1897-A 2021-11-09 08:24:00 2021-11-09 08:48:30 0 days 00:24:30   
8    AS762-A 2021-11-09 08:28:00 2021-11-09 08:52:00 0 days 00:24:00   
25   UA295-A 2021-11-09 07:35:30 2021-11-09 08:01:30 0 days 00:26:00   
0   AA1322-A 2021-11-09 07:58:00 2021-11-09 08:11:30 0 days 00:13:30   
12  UA1575-A 2021-11-09 07:25:00 2021-11-09 07:50:00 0 days 00:25:00   
6   AS3327-A 2021-11-09 07:39:00 2021-11-09 08:04:00 0 days 00:25:00   
30  UA5243-D 2021-11-09 11:42:00 2021-11-09 11:56:00 0 days 00:14:00   
39  UA5976-A 2021-11-09 07:22:00 2021-11-09 07:34:00 0 days 00:12:00   
27   UA368-A 202

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [ ]:
# print(len(overlap_arrivals))
# print(df_all[:5])

## Departure Flights

In [176]:
# df = pd.read_csv ('output/real-west-all-terminals/states.csv', header = 0)
departures = df.groupby(by=['callsign']).filter(lambda x: (x['takeoff'].max() == True).any())
departures = departures[['time', 'callsign']]
# print(arrivals[arrivals['callsign'] == 'AA1260-D'])
print(len(departures))
departures['time'] = departures['time'].astype('datetime64[ns]')

depart_taxi_time = departures.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
depart_taxi_time['taxi_time'] = depart_taxi_time['time']['max'] - depart_taxi_time['time']['min']
depart_taxi_time['taxi_time_seconds'] = depart_taxi_time['taxi_time'].dt.total_seconds()
print(len(depart_taxi_time))
# print(depart_taxi_time[:10])

avg_total_ticks = depart_taxi_time["taxi_time_seconds"].mean()
print(avg_total_ticks)

4271
102
1226.1764705882354


In [179]:
departures_before = df_before.groupby(by=['callsign']).filter(lambda x: (x['takeoff'].max() == True).any())
departures_before = departures_before[['time', 'callsign']]
# print(arrivals[arrivals['callsign'] == 'AA1260-D'])
# print(len(departures_before))
departures_before['time'] = departures_before['time'].astype('datetime64[ns]')

depart_taxi_time_before = departures_before.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
depart_taxi_time_before['taxi_time'] = depart_taxi_time_before['time']['max'] - depart_taxi_time_before['time']['min']
depart_taxi_time_before['taxi_time_seconds'] = depart_taxi_time_before['taxi_time'].dt.total_seconds()
print(len(depart_taxi_time_before))
# print(depart_taxi_time_before[:10])

avg_total_ticks_before = depart_taxi_time_before["taxi_time_seconds"].mean()
print(avg_total_ticks_before)

122
1606.4754098360656


In [165]:
overlap_departures = pd.merge(depart_taxi_time_before, depart_taxi_time, on="callsign")
print("departure")
print("before: " + str(overlap_departures['taxi_time_seconds_x'].mean()))
print("after: " + str(overlap_departures['taxi_time_seconds_y'].mean()))

departure
before: 1353.8709677419354
after: 1151.9354838709678


In [164]:
depart_only_before = depart_taxi_time_before.merge(depart_taxi_time, on=['callsign'], 
                   how='left', indicator=True)
depart_only_before = depart_only_before[depart_only_before['_merge'] == 'left_only']
print(len(depart_only_before))
print(depart_only_before['callsign'][:5])

depart_only_after = depart_taxi_time.merge(depart_taxi_time_before, on=['callsign'], 
                   how='left', indicator=True)
depart_only_after = depart_only_after[depart_only_after['_merge'] == 'left_only']
print(len(depart_only_after))
print(depart_only_after['callsign'][:5])

28
14    AC8839-D
45     B6416-D
48    DL1390-D
54    DL2490-D
60     DL664-D
Name: callsign, dtype: object
9
2       AA16-D
15     AC760-D
29    AS1952-D
43    AS3488-D
46     AS427-D
Name: callsign, dtype: object


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Arrivals for 7/10/15 hours

In [309]:
df = pd.read_csv ('output/real-west-all-terminals/states.csv', header = 0)
df['time'] = df['time'].astype('datetime64[ns]')

df_before = pd.read_csv ('/Users/lolaliu/Desktop/airport-simulation/output/real-west-all-terminals/states.csv', header = 0)
df_before['time'] = df_before['time'].astype('datetime64[ns]')

In [310]:
from datetime import datetime

seven_hours = pd.to_datetime('14:00:00')
ten_hours = pd.to_datetime('17:00:00')
fifteen_hours = pd.to_datetime('22:00:00')

time_arr = [seven_hours, ten_hours, fifteen_hours]

In [312]:
print("Before")
print("total flights: " + str(len(df_before.groupby(by=['callsign']).count())) + '\n')

for t in time_arr:
    print("Time range from 7:00:00 to " + str(t).split(" ")[1])
    arrivals_before = df_before.groupby(by=['callsign']).filter(lambda x: x.takeoff.max() == False and x.time.max() <= t)
    arrivals_before = arrivals_before[["time", "callsign"]]

    arrival_taxi_time_before = arrivals_before.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
    arrival_taxi_time_before['taxi_time'] = arrival_taxi_time_before['time']['max'] - arrival_taxi_time_before["time"]["min"]
    arrival_taxi_time_before['taxi_time_seconds'] = arrival_taxi_time_before['taxi_time'].dt.total_seconds()
    print("arrival flights: " + str(len(arrival_taxi_time_before)))

    sort_before = arrival_taxi_time_before.sort_values(by=[('time', 'max'), 'taxi_time_seconds'], ascending=False)
    # print(sort_before[:10])
    print("Average arrival time: " + str(sort_before["taxi_time_seconds"].mean()) + '\n')

Before
total flights: 511

Time range from 7:00:00 to 14:00:00
arrival flights: 34
Average arrival time: 815.2941176470588

Time range from 7:00:00 to 17:00:00
arrival flights: 37
Average arrival time: 2428.3783783783783

Time range from 7:00:00 to 22:00:00
arrival flights: 38
Average arrival time: 3480.0



In [311]:
print("After")

for t in time_arr:
    print("Time range from 7:00:00 to " + str(t).split(" ")[1])
    arrivals = df.groupby('callsign').filter(lambda x: x.takeoff.max() == False and x.time.max() <= t)
    arrivals = arrivals[['time', 'callsign']]

    arrival_taxi_time = arrivals.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
    arrival_taxi_time['taxi_time'] = arrival_taxi_time['time']['max'] - arrival_taxi_time['time']['min']
    arrival_taxi_time['taxi_time_seconds'] = arrival_taxi_time['taxi_time'].dt.total_seconds()
    print("arrival flights: " + str(len(arrival_taxi_time)))

    sort = arrival_taxi_time.sort_values(by=[('time', 'max'), 'taxi_time_seconds'], ascending=False)
    print("Average arrival time: " + str(sort["taxi_time_seconds"].mean()))
#     print(sort[:10])
    print()
    

After
Time range from 7:00:00 to 14:00:00
arrival flights: 102
Average arrival time: 1160.8823529411766

Time range from 7:00:00 to 17:00:00
arrival flights: 165
Average arrival time: 984.1818181818181

Time range from 7:00:00 to 22:00:00
arrival flights: 197
Average arrival time: 1335.5329949238578



## Departures for 7/10/15 hours

In [313]:
print("Departure Before")
print("total flights: " + str(len(df_before.groupby(by=['callsign']).count())) + '\n')
for t in time_arr:
    print("Time range from 7:00:00 to " + str(t).split(" ")[1])
    departures_before = df_before.groupby(by=['callsign']).filter(lambda x: x.takeoff.max() == True and x.time.max() <= t)
    departures_before = departures_before[['time', 'callsign']]
    
    depart_taxi_time_before = departures_before.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
    depart_taxi_time_before['taxi_time'] = depart_taxi_time_before['time']['max'] - depart_taxi_time_before['time']['min']
    depart_taxi_time_before['taxi_time_seconds'] = depart_taxi_time_before['taxi_time'].dt.total_seconds()
    print("departure flights: " + str(len(depart_taxi_time_before)))
    # print(depart_taxi_time_before[:10])

    avg_total_ticks_before = depart_taxi_time_before["taxi_time_seconds"].mean()
    print("Average departure time: " + str(avg_total_ticks_before))
    print()

Departure Before
total flights: 511

Time range from 7:00:00 to 14:00:00
departure flights: 181
Average departure time: 2077.292817679558

Time range from 7:00:00 to 17:00:00
departure flights: 257
Average departure time: 2698.715953307393

Time range from 7:00:00 to 22:00:00
departure flights: 381
Average departure time: 3363.8582677165355



In [314]:
print("Departure After")
print("total flights: " + str(len(df.groupby(by=['callsign']).count())) + '\n')

for t in time_arr:
    print("Time range from 7:00:00 to " + str(t).split(" ")[1])
    departures = df.groupby(by=['callsign']).filter(lambda x: x.takeoff.max() == True and x.time.max() <= t)
    departures = departures[['time', 'callsign']]
    

    depart_taxi_time = departures.groupby(by=['callsign'], as_index=False).agg({'time': ['min', 'max']})
    depart_taxi_time['taxi_time'] = depart_taxi_time['time']['max'] - depart_taxi_time['time']['min']
    depart_taxi_time['taxi_time_seconds'] = depart_taxi_time['taxi_time'].dt.total_seconds()
    print("departure flights: " + str(len(depart_taxi_time)))
    # print(depart_taxi_time[:10])

    avg_total_ticks = depart_taxi_time["taxi_time_seconds"].mean()
    print("Average departure time: " + str(avg_total_ticks))
    print()
    

Departure After
total flights: 389

Time range from 7:00:00 to 14:00:00
departure flights: 126
Average departure time: 1247.3809523809523

Time range from 7:00:00 to 17:00:00
departure flights: 144
Average departure time: 1194.7916666666667

Time range from 7:00:00 to 22:00:00
departure flights: 192
Average departure time: 1234.375

